In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By # 웹사이트의 구성요소를 선택하기 위해 By 모듈을 불려옵니다
from selenium.webdriver.support.ui import WebDriverWait # 웹페이지가 전부 로드될때까지 기다리는 (Explicitly wait) 기능을 하는 모듈입니다
from webdriver_manager.chrome import ChromeDriverManager # 크롬에서 크롤링을 하기 위해, 웹 드라이버를 설치하는 모듈입니다
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC # 크롬의 어떤 부분의 상태를 확인하는 모듈입니다
import time # 정해진 시간만큼 기다리게 하기 위한 패키지입니다
from selenium.webdriver.common.keys import Keys

In [8]:
Tsherpa_url= "https://edu.tsherpa.co.kr/LectureInfo/LectureReviews2"


In [9]:
driver = webdriver.Chrome()
driver.get(Tsherpa_url)

In [10]:
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mainForm"]'))
    )
finally:
    pass

In [11]:
fnm = '' # 맨 첫번째 상호 이름입니다. 다음 페이지에서도 똑같은 상호가 나온다면, 다음 페이지가 없다고 인식하고, brk 값이 바뀌면서 반복문이 종료되지요
brk = 0
nxt = 2
res = [] # 결과 파일은 판다스 데이터프레임으로 입력할겁니다
page_count = 0
stop_code = True

while True:
    
    page_count += 1
    brk += 1

    for i in range(1,11):
        
        time.sleep(0.5)
        hour = driver.find_element(by='xpath',value = f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a/span').text
        
        if driver.find_element(By.XPATH, f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a').is_displayed():
            driver.find_element(By.XPATH, f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a').send_keys(Keys.ENTER)
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[-1])

            nm = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[1]/span/a')
            yr = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[2]')
            scp = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[1]')
            desc = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[3]/td')

        year = yr[0].text.split('|')[1].strip()
        year = year.split('.')[0].strip()

      
        if int(year) <= 2017:
            print('stop')
            stop_code = False
            break

        res.append([nm[0].text, hour, yr[0].text.split('|')[1].strip(), scp[0].text, desc[0].text])

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    if stop_code != True:
        print(brk)
        driver.quit()
        break


    # 다음 페이지로 넘어가는 코드입니다    
    if page_count != 11:
        driver.find_element(By.XPATH, f'//*[@id="mainForm"]/div[3]/a[{page_count+nxt}]').send_keys(Keys.ENTER)

    else:
       driver.find_element(By.XPATH, f'//*[@id="mainForm"]/div[3]/a[12]').send_keys(Keys.ENTER)
       page_count = 0

    time.sleep(2.5)



            





stop
1


In [12]:
df = pd.DataFrame(res)
df.to_csv('./res_Tsherpa.csv', index=False) # 데이터가 실시간으로 저장되도록 합니다
df

""
